In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
import numpy as np
import tensorflow as tf
import os
import cv2
from PIL import Image
import time
import glob

#Directory locations for the code
image_path = "Data\\img_align_celeba"

#Cropped Output
output_path = "Data\\cropped_faces"
if not os.path.exists(output_path):
        os.mkdir(output_path)
        
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

KeyboardInterrupt: 

In [ ]:
#####Cropping just the face of the celebrities#####
face_cascade = "Data\\xml\\haarcascade_frontalface_default.xml"

def crop_and_save_images(cascade, imgname, output_path):
    img = cv2.imread(os.path.join(image_path, imgname))
    for i, face in enumerate(cascade.detectMultiScale(img)):
        #start = time.time()
        x, y, w, h = face
        sub_face = img[y:y + h, x:x + w]
        img_path = os.path.join(output_path, imgname)
        cv2.imwrite(img_path, sub_face)
        #if i%1000 == 0:
            #end = time.time() - start
            #print(i,"took", end, "seconds")
            #print(i, "Cropped")
            #start = time.time()
            
#Cropping the images to contain only faces
cascade = cv2.CascadeClassifier(face_cascade)
for f in [f for f in os.listdir(image_path) if os.path.isfile(os.path.join(image_path, f))]:
    crop_and_save_images(cascade, f, output_path)

In [ ]:
#some image manipulation functions
def resizer(path,scale="L",resize_x=28,resize_y=28):
    return np.array(Image.open(path).convert(scale).resize((resize_x,resize_y), Image.ANTIALIAS)).ravel().tolist()

def images_from_dirs(dirs,start,end,scaled = False):
    train_img = 255 - np.array([resizer(i,"L",28,28) for i in glob.glob(dirs)[start:end]],dtype='float32')
    return train_img if scaled else train_img/255.0

In [ ]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
 
    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=2)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=2)
    loss = tf.losses.softmax_cross_entropy(
      onehot_labels=onehot_labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
training_data_start_index = 0
training_data_end_index = 200000

In [ ]:
def read_data(training_data_start_index,training_data_end_index):
    #Eyeglasses label
    feature = pd.read_csv("Data/glasses.csv")
    eye_glasses = feature[["Images","Eyeglasses"]]

    Y_label = feature.Eyeglasses
    Y_label = np.array([i if i == 1 else 0 for i in list(Y_label)])

    #Importing images as numpy objects
    images = images_from_dirs(output_path + "\\*.jpg",training_data_start_index,training_data_end_index)
    data = {"x":images,"y":Y_label[training_data_start_index:training_data_end_index]}
    return data

In [ ]:
data = read_data(training_data_start_index,training_data_end_index)

In [ ]:
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="tmp/mnist_convnet_model")
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": data["x"]},
    y=data["y"],
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=200000,
    hooks=[logging_hook])

In [ ]:
data = read_data(20000,22000)

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": data["x"]},
    y=data["y"],
    num_epochs=10,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

In [ ]:
array = [1,2,2]

In [ ]:
from scipy.stats import gaussian_kde
a = gaussian_kde(array)
result = [a.evaluate(i) for i in range(len(array))]
print(np.array(result))